## 정리

지금까지 다룬 내용들을 정리해보자.  
먼저 신경망에는 가중치와 편향이 있고 이 매개변수들을 훈련 데이터에 적응하도록 조정하는 과정을 학습이라 한다.  
1. 전체 학습 데이터 중 일부를 무작위로 선별하여 신경망의 입력으로 사용한다. - batch
2. 입력한 데이터에 대한 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 이때 기울기는 손실 함수의 최솟값을 구하는 방향을 제시.
3. 매개변수를 기울기 방향으로 아주 조금 갱신한다. - learning rate
4. 1 ~ 3을 반복

이때 데이터를 무작위로 선정하기 때문에 확률적 경사 하강법(SGD - Stochastic Gradient Descent)라고 부른다.

In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [3]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [4]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [5]:
print(net.params['W1'].shape)
print(net.params['b1'].shape)
print(net.params['W2'].shape)
print(net.params['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


In [6]:
from mnist import load_mnist

In [7]:
x_train, y_train, x_test, y_test = load_mnist(pickle_path="/data/Datasets/MNIST/mnist.pkl", normalize=True, one_hot_label=True)

In [8]:
train_loss_list = []
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

배치 크기가 100이므로 매번 60000개의 훈련 데이터에서 임의로 100개의 데이터를 추려낸다.  
그리고 배치 데이터들을 대상으로 확률적 경사 하강법을 수행해 매개변수를 갱신한다.

경사 하강법으로 갱신하는 횟수를 10000번으로 설정하고 갱신할 때마다 훈련 데이터에 대한 손실함수를 계산하고 리스트에 그 값을 저장.

In [9]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [10]:
for i in range(iters_num):
    print(f"Iter {i}")
    batch_set = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_set]
    y_batch = y_train[batch_set]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, y_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)

Iter 0
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
Iter 10
Iter 11
Iter 12
Iter 13
Iter 14
Iter 15
Iter 16
Iter 17
Iter 18
Iter 19
Iter 20
Iter 21
Iter 22
Iter 23
Iter 24
Iter 25
Iter 26
Iter 27
Iter 28
Iter 29
Iter 30
Iter 31
Iter 32
Iter 33
Iter 34
Iter 35
Iter 36
Iter 37
Iter 38
Iter 39
Iter 40
Iter 41
Iter 42
Iter 43
Iter 44
Iter 45
Iter 46
Iter 47
Iter 48
Iter 49
Iter 50
Iter 51
Iter 52
Iter 53
Iter 54
Iter 55
Iter 56
Iter 57
Iter 58
Iter 59
Iter 60
Iter 61
Iter 62
Iter 63
Iter 64
Iter 65
Iter 66
Iter 67
Iter 68
Iter 69
Iter 70
Iter 71
Iter 72
Iter 73
Iter 74
Iter 75
Iter 76
Iter 77
Iter 78
Iter 79
Iter 80
Iter 81
Iter 82
Iter 83
Iter 84
Iter 85
Iter 86
Iter 87
Iter 88
Iter 89
Iter 90
Iter 91
Iter 92
Iter 93
Iter 94
Iter 95
Iter 96
Iter 97
Iter 98
Iter 99
Iter 100
Iter 101
Iter 102
Iter 103
Iter 104
Iter 105
Iter 106
Iter 107
Iter 108
Iter 109
Iter 110
Iter 111
Iter 112
Iter 113
Iter 114
Iter 115
Iter 116
Iter 117
Iter 118
Iter 119
Iter 120
Iter 121
Iter 122
Ite

KeyboardInterrupt: 

학습 데이터의 배치 데이터에 대한 손실 함수값이 감소. 이것이 신경망이 잘 학습하고 있다는 지표이지만, 실제 현실 서비스에 적용할 범용성을 갖추고 있는지는 아직 모른다.

신경망 학습에서는 학습 데이터외의 데이터를 올바르게 인식하는지 확인해야 하는데 이를 다른 말로 오버피팅을 일으키지 않았는지 확인하는 것이다.  
오버피팅이 발생하면 학습 데이터에 포함된 이미지만 제대로 구분하고, 그렇지 않은 데이터는 식별하지 못함.  
따라서 테스트 데이터 대상으로 정확도를 기록해보자.

여기서는 1 epoch별로 테스트 데이터에 대한 정확도를 기록할 것이다.
> 1 epoch은 학습 데이터를 모두 소진했을 때의 횟수에 해당. 10000개의 학습 데이터를 100개의 배치로 학습한다면 SGD로 100회 반복하면 모든 학습 데이터를 소진한 것이 된다. 따라서 100회 반복 = 1 epoch

In [ ]:
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_sizeeeezezezezeze / batch_size, 1)

In [ ]:
for i in range(iters_num):
    batch_mask = np.random.choicececececece(train_size, batch_size)
    x_batch, y_batch = x_train[batch_mask], y_train[batch_mask]
    
    grad = network.numerical_gradient(x_batch, y_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, y_train)
        test_acc = network.accuracy(x_test, y_test)
        
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        print(f"train acc, test acc | {train_acc}, {test_acc}")